In [10]:
from word_embeddings import WB_Model
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.similarities import SparseMatrixSimilarity
from gensim.utils import simple_preprocess
from transformers import pipeline
import os

In [11]:
wb_model = WB_Model('../../data/dre.json')
wb_model.load_model('models/dr.model')

In [27]:
sentences = wb_model.sentences

metadata = wb_model.metadata

In [52]:
dictionary = Dictionary(sentences)

corpus_bow = [dictionary.doc2bow(sent) for sent in sentences]

tfidf_model = TfidfModel(corpus_bow, normalize=True)

index = SparseMatrixSimilarity(tfidf_model[corpus_bow], num_docs= len(corpus_bow), num_terms=len(dictionary))

In [46]:
query = 'O que implica a declaração que o conselheiro Joaquim Jorge de Pinho Campinos  apresentou, em 12 de Agosto de 1985?'
query_tokens = simple_preprocess(query)

In [15]:
def get_text(id):
    file_path = f'../../data/texts/{id}.txt'
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    return ''

In [53]:
## Caso 1
most_similar = wb_model.get_most_similar(query,10)

context_case1 = '\n'.join(get_text(id) for id, note, score in most_similar )

for id, notes, score in most_similar:
    print(id, notes, score)

2394463 Decreto de 29 de março, dando provimento no recurso n.º 13425, em que era recorrente João Pedroso de Freitas 0.8194894214982434
1562089 Declaração de contumácia no processo n.º 8/03.4GCRDD, em que é arguido José Francisco Grazina de Deus 0.8178253218124993
2432450 Considera sem efeito o decreto de 15 de Maio de 1919, que havia demitido Leopoldo Rui Gomes de Oliveira de primeiro secretário de legação - Reintegra e considera na situação de disponibilidade o referido funcionário 0.8177859401468984
2401916 Rectificação ao decreto n.º 398, de 30 de Março, que resolveu o recurso n º 14447, em que era recorrente Aníbal de Assunção Soares 0.8177524990576575
2394327 Decreto de 17 de janeiro, rejeitando o recurso n.º 13055, em que era recorrente Manuel Antonio Pereira, de Borba 0.8162497623867817
202145 Declara ter sido rectificado o Decreto n.º 101/81, de 31 de Julho, que nomeia o ministro plenipotenciário de 2.ª classe Luís de Vasconcelos Pimentel Quartin Bastos embaixados de Portugal 

In [54]:
## Caso 2
query_bow = dictionary.doc2bow(query_tokens)
tfidf_query = tfidf_model[query_bow]

sims = index[tfidf_query]

sims_ordered = sorted(enumerate(sims), key= lambda item: item[1], reverse = True)

context_case2 = '\n'.join(get_text(metadata[index]['id']) for index, _ in sims_ordered[:10])

for index, sim in sims_ordered[:10]:
    print(metadata[index]['id'], metadata[index]['notes'], sim)

210336 Designa o Prof. Doutor Joaquim Jorge de Pinho Campinos vogal da Comissão Constitucional. 0.4568582
11 De ter sido apresentada declaração escrita de renúncia das suas funções de juiz do Tribunal Constitucional pelo conselheiro Joaquim Jorge de Pinho Campinos. 0.43540695
2485271 De ter sido apresentada declaração escrita de renúncia das suas funções de juiz do Tribunal Constitucional pelo conselheiro Joaquim Jorge de Pinho Campinos 0.43540695
11601 Delega competências do Primeiro-Ministro, Mário Soares no Ministro sem Pasta, Joaquim Jorge de Pinho Campinos. 0.39206162
11621 Delega competências do Primeiro-Ministro, Mário Soares no Ministro sem Pasta, Joaquim Jorge de Pinho Campinos. 0.39206162
51439 DECLARA QUE O CONSELHEIRO MÁRIO DE BRITO APRESENTOU DECLARAÇÃO ESCRITA DE RENÚNCIA DAS SUAS FUNÇÕES DE JUIZ DO TRIBUNAL CONSTITUCIONAL, A QUAL NAO DEPENDE DE ACEITAÇÃO E IMPLICA A CESSACAO IMEDIATA DE FUNÇÕES. 0.3820706
11613 Delega competências do Primeiro-Ministro, Mário Soares no Mi

In [56]:
models = [
    "mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt",
    "lfcc/bert-portuguese-squad",
    "eraldoluis/faquad-bert-base-portuguese-cased"
]

for index, model in enumerate(models):
    question_answerer = pipeline("question-answering", model=model)
    result1 = question_answerer(question=query, context=context_case1)
    result2 = question_answerer(question=query, context=context_case2)

    print(f"# CASO 1: MODELO {index}: {result1['answer']}")
    print(f"# CASO 2: MODELO {index}: {result2['answer']}")
    

Some weights of the model checkpoint at mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# CASO 1: MODELO 0: Decreto do Presidente da República
# CASO 2: MODELO 0: cessação imediata de funções
# CASO 1: MODELO 1: situação de disponibilidade
# CASO 2: MODELO 1: renúncia às suas funções de juiz do Tribunal Constitucional
# CASO 1: MODELO 2: inexactidão
# CASO 2: MODELO 2: renúncia das suas  funções de juiz do Tribunal Constitucional
